In [740]:
%run "Segmenting and Clustering Neighborhoods in Toronto - Part 2.ipynb"

The final part of this assignment starts by reading the coordinates of Toronto from its page on Wikipedia. Then, we add markers of the neighborhoods (from the previous part) in the map of Toronto:

In [744]:
import folium
import wikipedia
import re
import requests
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
import random

toronto_wiki_page = wikipedia.page("Toronto")
toronto_wiki_data = toronto_wiki_page.html()
toronto_coordinates_string = re.search(r'<span class="geo">(.*?)</span></span></span></a>', toronto_wiki_data).group(1)
[latitude,longitude] = [float(item) for item in toronto_coordinates_string.split(";")]

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10, width='70%', height='70%')

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_neighborhood['Latitude'], 
                                           df_toronto_neighborhood['Longitude'], 
                                           df_toronto_neighborhood['Borough'], 
                                           df_toronto_neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now, let's use the FourSquare API to gather information about Toronto venues. First, let's initialize the FourSquare credentials:

In [745]:
CLIENT_ID = '2REBOOS3YFKNJDH4PVUAELAGFULDILSOVRNYA2OXOP3TH0GC' # your Foursquare ID
CLIENT_SECRET = 'HUYA1HXT0WXNO2UXSHYT4YD1E3RH2AID0IVAUBT41CTQDZR5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Next, as done in the tutorial notebook shared in the course, let's create a function that retrieve information about the venues per neighborhood:

In [748]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We execute the function above to actually gather the Toronto venues' information:

In [749]:
toronto_venues = getNearbyVenues(names=df_toronto_neighborhood['Neighborhood'],
                                   latitudes=df_toronto_neighborhood['Latitude'],
                                   longitudes=df_toronto_neighborhood['Longitude']
                                  )

In [750]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,25,25,25,25,25,25
...,...,...,...,...,...,...
Willowdale,39,39,39,39,39,39
Woburn,4,4,4,4,4,4
Woodbine Heights,9,9,9,9,9,9


Now let's do some work in the Pandas dataframe to investigate what type of venues are present in each neighborhood. More specifically, I want to check how the neighborhoods are classified in terms of options related to food - restaurants, bars, coffee shops, etc. So let's first reduce our data set to contain only the neighborhoods

In [786]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

for category in toronto_onehot.columns:
    food_flag = 0
    for label in ["Restaurant","Place","Bar","Coffee"]:
        if label in category:
            food_flag = 1
    if food_flag == 0:
        toronto_onehot.drop([category], axis=1, inplace=True)



# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot = toronto_onehot[ ['Neighborhood'] + [ col for col in toronto_onehot.columns if col != 'Neighborhood' ] ]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean()*100
toronto_grouped = toronto_grouped.loc[(toronto_grouped!=0).any(axis=1)]
toronto_grouped




,American Restaurant,Asian Restaurant,Bar,Beer Bar,Belgian Restaurant,Brazilian Restaurant,Burrito Place,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Hotel Bar,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Snack Place,Soup Place,Sports Bar,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,20.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,12.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,25.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,12.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.000000,5.000000,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,25.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Bedford Park / Lawrence Manor East,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,8.000000,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.000000,8.0,0.000000,4.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.000000,8.000000,0.0,0.0,0.0,8.000000,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,4.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westmount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.285714,0.0,14.285714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,28.571429,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,14.285714,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Wexford / Maryvale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,16.666667,0.0,0.0,0.0,0.0,0.000000,0.0,0.

With the above information about the venues per neighborhood, we can now cluster the different neighborhoods, using sklearn' KMeans. Let's consider 6 clusters:

In [787]:
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=random.randint(0,1000)).fit(toronto_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 0, 1, 1, 1, 1, 3, 1, 1,
       1, 1, 5, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 4, 1, 5, 1, 0, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 0, 5, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 5, 1])

Next, we add the latitude, longitude and cluster information to our "Toronto venues" Pandas dataframe. This will allow us to plot the different clusters in a map:

In [788]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped.merge(df_toronto_neighborhood,
                                       left_on='Neighborhood', 
                                       right_on='Neighborhood',)

toronto_merged = toronto_merged.set_index('Neighborhood')
toronto_merged

,Cluster Labels,American Restaurant,Asian Restaurant,Bar,Beer Bar,Belgian Restaurant,Brazilian Restaurant,Burrito Place,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Hotel Bar,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Ramen Restaurant,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Snack Place,Soup Place,Sports Bar,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,20.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,M1S,Scarborough,43.7942003,-79.2620294
Alderwood / Long Branch,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,25.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,12.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,M8W,Etobicoke,43.6024137,-79.5434841
Bathurst Manor / Wilson Heights / Downsview North,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.000000,5.000000,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,M3H,North York,43.7543283,-79.4422593
Bayview Village,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,25.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,M2K,North York,43.7869473,-79.385975
Bedford Park / Lawrence Manor East,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.000000,8.0,0.000000,4.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.000000,8.000000,0.0,0.0,0.0,8.000000,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,4.0,0.0,0.0,0.000000,0.0,M5M,North York,43.7332825,-79.4197497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wexford / Maryvale,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,16.666667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,16.666667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000

In [789]:
# create map
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=10, width='70%', height='70%')

import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  list(toronto_merged.index.values), 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, let's analyze some of the clusters individually to understand how the neighborhoods were organized. The first cluster seems to concentrate the neighborhoods with most options of Pizza places:

In [790]:
clusters = []
for i in range(6):
    cluster = toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
    clusters.append(cluster.loc[:, (cluster != 0).any(axis=0)])

In [791]:
clusters[0]

,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Pizza Place,Portuguese Restaurant,Sandwich Place,Thai Restaurant,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,,,,,,,,,,
Alderwood / Long Branch,0.000000,12.500000,0.000000,0.000000,0.0,25.000000,0.0,12.500000,0.000000,M8W,Etobicoke,43.6024137,-79.5434841
Clarks Corners / Tam O'Shanter / Sullivan,7.142857,0.000000,7.142857,7.142857,0.0,14.285714,0.0,0.000000,7.142857,M1T,Scarborough,43.7816375,-79.3043021
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood,0.000000,14.285714,0.000000,0.000000,0.0,14.285714,0.0,0.000000,0.000000,M9C,Etobicoke,43.6435152,-79.5772008
Glencairn,0.000000,0.000000,0.000000,0.000000,25.0,25.000000,0.0,0.000000,0.000000,M6B,North York,43.709577,-79.4450726
Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens,0.000000,0.000000,0.000000,0.000000,0.0,25.000000,0.0,25.000000,0.000000,M9R,Etobicoke,43.6889054,-79.5547244
Parkview Hill / Woodbine Gardens,0.000000,0.000000,8.333333,0.000000,0.0,16.666667,0.0,0.000000,0.000000,M4B,East York,43.7063972,-79.309937
Victoria Village,0.000000,25.000000,0.000000,0.000000,0.0,25.000000,25.0,0.000000,0.000000,M4A,North York,43.7258823,-79.3155716
Westmount,14.285714,14.285714,0.000000,0.000000,0.0,28.571429,0.0,14.285714,0.000000,M9P,Etobicoke,43.696319,-79.5322424


Next, the second cluster contains the neighborhoods with a more diversified set of cuisine types - i.e. several restaurant types per neighborhood, without a clear prevalence of a given type:

In [792]:
clusters[1]

,American Restaurant,Belgian Restaurant,Brazilian Restaurant,Burrito Place,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hakka Restaurant,Hotel Bar,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Pizza Place,Poke Place,Poutine Place,Ramen Restaurant,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Snack Place,Soup Place,Sports Bar,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,20.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,M1S,Scarborough,43.7942003,-79.2620294
Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,5.000000,0.000000,10.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.000000,5.000000,0.0,0.0,0.0,5.000000,0.000000,0.0,0.0,0.0,5.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,M3H,North York,43.7543283,-79.4422593
Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,25.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,25.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,M2K,North York,43.7869473,-79.385975
Bedford Park / Lawrence Manor East,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,8.000000,0.0,4.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,4.000000,0.0,0.000000,0.0,0.0,0.0,4.000000,0.0,0.0,4.000000,0.000000,8.000000,0.000000,4.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.000000,8.000000,0.0,0.0,0.0,8.000000,0.000000,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,4.000000,0.0,0.000000,0.000000,0.0,M5M,North York,43.7332825,-79.4197497
Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.636364,5.454545,0.0,1.818182,0.0,0.0,0.0,0.0,1.818182,0.0,0.0,0.000000,0.0,1.818182,0.0,0.0,0.0,1.818182,0.0,0.0,1.818182,0.000000,3.636364,1.818182,1.818182,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,3.636364,0.0,0.0,0.0,0.000000,3.636364,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.818182,0.0,1.818182,0.000000,0.0,M5E,Downtown Toronto,43.6447708,-79.3733064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
University of Toronto / Harbord,0.0,0.0,0.0,0.0,0.0,0.0,2.857143,0.000000,0.000000,0.0,2.857143,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,2.857143,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,5.714286,5.714286,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,5.714286,0.0,0.0

The following cluster include a single neighborhood. It has a peculiarity: half of its total venues are fast food restaurants! Further analysis could indicate if this is due to too few venues in this neighborhood or if it indeed has a lot of restaurants of this type.

In [793]:
clusters[2]

,Fast Food Restaurant,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,,
Malvern / Rouge,50.0,M1B,Scarborough,43.8066863,-79.1943534


A final cluster interesting to check is the one below, containing the neighborhoods with prevalence of coffee shops:

In [799]:
clusters[5]

,Burrito Place,Chinese Restaurant,Coffee Shop,Falafel Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Salad Place,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Postal code,Borough,Latitude,Longitude
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,
Central Bay Street,0.000,1.5625,18.750000,1.5625,1.5625,1.5625,6.250,3.125,0.000,1.5625,1.5625,0.000,1.5625,1.5625,3.125,4.6875,3.125,1.5625,1.5625,1.5625,M5G,Downtown Toronto,43.6579524,-79.3873826
East Toronto,0.000,0.0000,33.333333,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.0000,0.0000,M4J,East York,43.685347,-79.3381065
Kennedy Park / Ionview / East Birchmount Park,0.000,0.0000,20.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.0000,0.0000,M1K,Scarborough,43.7279292,-79.2620294
Queen's Park / Ontario Provincial Government,3.125,0.0000,25.000000,0.0000,0.0000,0.0000,3.125,0.000,3.125,0.0000,0.0000,3.125,0.0000,0.0000,0.000,3.1250,0.000,0.0000,0.0000,0.0000,M7A,Downtown Toronto,43.6623015,-79.3894938
Woburn,0.000,0.0000,50.000000,0.0000,0.0000,25.0000,0.000,0.000,0.000,25.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.0000,0.0000,M1G,Scarborough,43.7709921,-79.2169174
